# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
J. Liu  ->  J. Liu  |  ['J. Liu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
R. Burn  ->  R. Burn  |  ['R. Burn']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']


Arxiv has 56 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.14025


extracting tarball to tmp_2501.14025...

 done.
Retrieving document from  https://arxiv.org/e-print/2501.14145


extracting tarball to tmp_2501.14145...

...

 done.


Found 154 bibliographic references in tmp_2501.14408/Euclid-TNG50.bbl.
Issues with the citations
syntax error in line 354: '=' expected
Retrieving document from  https://arxiv.org/e-print/2501.14494
extracting tarball to tmp_2501.14494...

 done.


J. Li  ->  J. Li  |  ['J. Li']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']


Found 67 bibliographic references in tmp_2501.14494/wdms.bbl.
Issues with the citations
syntax error in line 459: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.14355-b31b1b.svg)](https://arxiv.org/abs/2501.14355) | **The NCORES Program: Precise planetary masses, null results, and insight into the planet mass distribution near the radius gap**  |
|| D. J. Armstrong, et al. -- incl., <mark>R. Burn</mark> |
|*Appeared on*| *2025-01-27*|
|*Comments*| *Accepted to MNRAS*|
|**Abstract**|            NCORES was a large observing program on the ESO HARPS spectrograph, dedicated to measuring the masses of Neptune-like and smaller transiting planets discovered by the TESS satellite using the radial velocity technique. This paper presents an overview of the programme, its scientific goals and published results, covering 35 planets in 18 planetary systems. We present spectrally derived stellar characterisation and mass constraints for five additional TOIs where radial velocity observations found only marginally significant signals (TOI-510.01, $M_p=1.08^{+0.58}_{-0.55}M_\oplus$), or found no signal (TOIs 271.01, 641.01, 697.01 and 745.01). A newly detected non-transiting radial velocity candidate is presented orbiting TOI-510 on a 10.0d orbit, with a minimum mass of $4.82^{+1.29}_{-1.26}M_\oplus$, although uncertainties on the system architecture and true orbital period remain. Combining the NCORES sample with archival known planets we investigate the distribution of planet masses and compositions around and below the radius gap, finding that the population of planets below the gap is consistent with a rocky composition and ranges up to a sharp cut-off at $10M_\oplus$. We compare the observed distribution to models of pebble- and planetesimal-driven formation and evolution, finding good broad agreement with both models while highlighting interesting areas of potential discrepancy. Increased numbers of precisely measured planet masses in this parameter space are required to distinguish between pebble and planetesimal accretion.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.14408-b31b1b.svg)](https://arxiv.org/abs/2501.14408) | **Euclid preparation: Extracting physical parameters from galaxies with machine learning**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-01-27*|
|*Comments*| **|
|**Abstract**|            The Euclid mission is generating a vast amount of imaging data in four broadband filters at high angular resolution. This will allow the detailed study of mass, metallicity, and stellar populations across galaxies, which will constrain their formation and evolutionary pathways. Transforming the Euclid imaging for large samples of galaxies into maps of physical parameters in an efficient and reliable manner is an outstanding challenge. We investigate the power and reliability of machine learning techniques to extract the distribution of physical parameters within well-resolved galaxies. We focus on estimating stellar mass surface density, mass-averaged stellar metallicity and age. We generate noise-free, synthetic high-resolution imaging data in the Euclid photometric bands for a set of 1154 galaxies from the TNG50 cosmological simulation. The images are generated with the SKIRT radiative transfer code, taking into account the complex 3D distribution of stellar populations and interstellar dust attenuation. We use a machine learning framework to map the idealised mock observational data to the physical parameters on a pixel-by-pixel basis. We find that stellar mass surface density can be accurately recovered with a $\leq 0.130 {\rm \,dex}$ scatter. Conversely, stellar metallicity and age estimates are, as expected, less robust, but still contain significant information which originates from underlying correlations at a sub-kpc scale between stellar mass surface density and stellar population properties.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.14494-b31b1b.svg)](https://arxiv.org/abs/2501.14494) | **Identification of 30,000 White Dwarf-Main Sequence binaries candidates from Gaia DR3 BP/RP(XP) low-resolution spectra**  |
|| <mark>J. Li</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>J. Müller-Horn</mark>, <mark>R. Seeburger</mark> |
|*Appeared on*| *2025-01-27*|
|*Comments*| *30 pages, 12 figures, Submitted to ApJS*|
|**Abstract**|            White dwarf-main sequence (WDMS) binary systems are essential probes for understanding binary stellar evolution and play a pivotal role in constraining theoretical models of various transient phenomena. In this study, we construct a catalog of WDMS binaries using Gaia DR3's low-resolution BP/RP (XP) spectra. Our approach integrates a model-independent neural network for spectral modelling with Gaussian Process Classification to accurately identify WDMS binaries among over 10 million stars within 1 kpc. This study identify approximately 30,000 WDMS binary candidates, including ~1,700 high-confidence systems confirmed through spectral fitting. Our technique is shown to be effective at detecting systems where the main-sequence star dominates the spectrum - cases that have historically challenged conventional methods. Validation using GALEX photometry reinforces the reliability of our classifications: 70\% of candidates with an absolute magnitude $M_{G} > 7$ exhibit UV excess, a characteristic signature of white dwarf companions. Our all-sky catalog of WDMS binaries expands the available dataset for studying binary evolution and white dwarf physics and sheds light on the formation of WDMS.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.14025-b31b1b.svg)](https://arxiv.org/abs/2501.14025) | **Cataclysmic Variables in Triples: Formation Models and New Discoveries**  |
|| C. Shariat, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-01-27*|
|*Comments*| *Submitted to PASP. Comments are welcome*|
|**Abstract**|            The formation of cataclysmic variables (CVs) has long been modeled as a product of common envelope evolution (CEE) in isolated binaries. However, a significant fraction of intermediate-mass stars -- the progenitors of the white dwarfs (WDs) in CVs -- are in triples. We therefore investigate the importance of triple star dynamics in CV formation. Using Gaia astrometry and existing CV catalogs, we construct a sample of $\sim50$ CVs in hierarchical triples within 1 kpc of the Sun, containing main-sequence (MS) and WD tertiaries at separations of 100 - 30,000 au. We infer that at least 10% of CVs host wide tertiaries. To interpret this discovery, we evolve a population of 2000 triples using detailed three-body simulations, 47 of which become CVs. We predict that 20% of CVs in triples form without ever experiencing CEE, where the WD and donor are brought together by the eccentric Kozai-Lidov (EKL) mechanism after the formation of the WD. These systems favor larger donor stars and longer birth orbital periods (8-20 hrs) than typical CVs. Among systems that do undergo CEE, about half would not have interacted without the presence of the tertiary. Triple formation channels both with and without CEE require initially wide inner orbits ($\gtrsim 1$ au), which in turn require larger tertiary separations to be stable. Consistent with this prediction, we find that the observed Gaia CV triples have wider separations on average than normal wide binaries selected in the same way. Our work underscores the importance of triples in shaping interacting binary populations including CVs, ultracompact binaries, and low-mass X-ray binaries.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.14145-b31b1b.svg)](https://arxiv.org/abs/2501.14145) | **Resolution requirements for numerical modeling of neutrino quantum kinetics**  |
|| H. Nagakura, M. Zaizen, <mark>J. Liu</mark>, L. Johns |
|*Appeared on*| *2025-01-27*|
|*Comments*| *11 pages, 10 figures*|
|**Abstract**|            Neutrino quantum kinetics is a rapidly evolving field in computational astrophysics, with a primary focus on collective neutrino oscillations in core-collapse supernovae and post-merger phases of binary neutron star mergers. In recent years, there has been considerable debate concerning resolution dependence in numerical simulations. In this paper, we conduct a comprehensive resolution study in both angular- and spatial directions by using two independent schemes of quantum kinetic neutrino transport: finite volume and pseudospectral methods. We complement our discussion by linear stability analysis including inhomogeneous modes. Our result suggests that decreasing spatial resolutions underestimates the growth of flavor instability, and then leads to wrong asymptotic states of flavor conversions, which potentially has a critical impact on astrophysical consequences. We further delve into numerical results of low resolution simulations, that reveals the underlying mechanism responsible for numerical artifacts caused by insufficient resolutions. This study settles the debate on requirements of resolutions and serves as a guideline for numerical modeling of quantum kinetic neutrino transport.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.14247-b31b1b.svg)](https://arxiv.org/abs/2501.14247) | **A flaring radio counterpart to a fast radio burst reveals a newborn magnetized engine**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2025-01-27*|
|*Comments*| *37 pages including 14 figures. Comments are welcome*|
|**Abstract**|            Fast Radio Bursts (FRBs) are energetic millisecond radio bursts at cosmological distances, whose underlying engine is not identified. Among a sub-population that emit repeated radio bursts, a handful were associated with a persistent radio source (PRS) whose origin is unknown. Here we report the discovery of a compact flaring radio source (FRS) associated with a newly-active repeating FRB within one month after the first radio burst was detected. Its temporal and spectral characteristics differ from those of the PRSs but are similar to those of engine-powered supernovae and low-luminosity active galactic nuclei. We detected a spectral peak around $1.6\pm0.2$ GHz that is consistent with synchrotron self-absorption. Assuming equipartition, the magnetic field strength in the FRS is larger than the line-of-sight component constrained from the FRB Faraday rotation, suggesting a highly magnetized engine. The radius of the FRS is constrained to be $\sim0.03$ pc and the minimum total energy is $\sim~6.2\times{10}^{47}~{\rm ergs~}$. This FRS reveals the birth of a highly magnetized FRB engine, and hints that PRSs associated with other active FRBs may be in the later stage of evolution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.14355/./TPF_Gaia_TOI-510_TIC238086647_S6.png', 'tmp_2501.14355/./combined_rvphaseplot_041224.png', 'tmp_2501.14355/./SR_DM_NCORESonly_080724.png']
copying  tmp_2501.14355/./TPF_Gaia_TOI-510_TIC238086647_S6.png to _build/html/
copying  tmp_2501.14355/./combined_rvphaseplot_041224.png to _build/html/
copying  tmp_2501.14355/./SR_DM_NCORESonly_080724.png to _build/html/
exported in  _build/html/2501.14355.md
    + _build/html/tmp_2501.14355/./TPF_Gaia_TOI-510_TIC238086647_S6.png
    + _build/html/tmp_2501.14355/./combined_rvphaseplot_041224.png
    + _build/html/tmp_2501.14355/./SR_DM_NCORESonly_080724.png
found figures ['tmp_2501.14408/./Figures/models_all.png', 'tmp_2501.14408/./Figures/histograms_tot.png', 'tmp_2501.14408/./Figures/TNG539333_O5_good.png']
copying  tmp_2501.14408/./Figures/models_all.png to _build/html/
copying  tmp_2501.14408/./Figures/histograms_tot.png to _build/html/
copying  tmp_2501.14408/./Figures/TNG539333_O5_good.png to _build/html/
exp

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\ms}{m s^{-1}}$
$\newcommand{\mss}{\mbox{m s^{-2}}}$
$\newcommand{\masy}{mas yr^{-1}}$
$\newcommand{\mpl}{\mbox{M_{p}}}$
$\newcommand{\rpl}{\mbox{R_{p}}}$
$\newcommand{\mstar}{\mbox{M_{\star}}}$
$\newcommand{\rstar}{\mbox{R_{\star}}}$
$\newcommand{\mjup}{\mbox{M_{\rm Jup}}}$
$\newcommand{\rjup}{\mbox{R_{\rm Jup}}}$
$\newcommand{\msun}{\mbox{M_{\odot}}}$
$\newcommand{\rsun}{\mbox{R_{\odot}}}$
$\newcommand{\rearth}{R_{\oplus}}$
$\newcommand{\mearth}{M_{\oplus}}$
$\newcommand{\gccc}{g cm^{-3}}$
$\newcommand{\ergscm}{erg s^{-1}cm^{-2}}$
$\newcommand{\vsini}{v\sin{i}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\feh}{\mbox{\rm[Fe/H]}}$
$\newcommand{\ymg}{\mbox{\rm[Y/Mg]}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\rprs}{\mbox{R_{p}/R_{s}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\jv}{\textcolor[RGB]{200,20,50}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The NCORES Program: Precise planetary masses, null results, and insight into the planet mass distribution near the radius gap$\thanks{Based on observations collected at the European Southern Observatory under ESO programme 1102.C-0249 (PI: Armstrong). This paper includes data gathered with the 6.5 meter Magellan Telescopes located at Las Campanas Observatory, Chile.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.14355-b31b1b.svg)](https://arxiv.org/abs/2501.14355)<mark>Appeared on: 2025-01-27</mark> -  _Accepted to MNRAS_

</div>
<div id="authors">

D. J. Armstrong, et al. -- incl., <mark>R. Burn</mark>

</div>
<div id="abstract">

**Abstract:** NCORES was a large observing program on the ESO HARPS spectrograph, dedicated to measuring the masses of Neptune-like and smaller transiting planets discovered by the _TESS_ satellite using the radial velocity technique. This paper presents an overview of the programme, its scientific goals and published results, covering 35 planets in 18 planetary systems. We present spectrally derived stellar characterisation and mass constraints for five additional TOIs where radial velocity observations found only marginally significant signals (TOI-510.01, $M_p = 1.08^{+0.58}_{-0.55}M_\oplus$ ), or found no signal (TOIs 271.01, 641.01, 697.01 and 745.01). A newly detected non-transiting radial velocity candidate is presented orbiting TOI-510 on a 10.0d orbit, with a minimum mass of $4.82^{+1.29}_{-1.26}M_\oplus$ , although uncertainties on the system architecture and true orbital period remain. Combining the NCORES sample with archival known planets we investigate the distribution of planet masses and compositions around and below the radius gap, finding that the population of planets below the gap is consistent with a rocky composition and ranges up to a sharp cut-off at $10M_\oplus$ . We compare the observed distribution to models of pebble- and planetesimal-driven formation and evolution, finding good broad agreement with both models while highlighting interesting areas of potential discrepancy. Increased numbers of precisely measured planet masses in this parameter space are required to distinguish between pebble and planetesimal accretion.

</div>

<div id="div_fig1">

<img src="tmp_2501.14355/./TPF_Gaia_TOI-510_TIC238086647_S6.png" alt="Fig1" width="100%"/>

**Figure 1. -** _TESS_ field of view around TOI-510 in Sector 6, generated with \texttt{tpfplotter}. Known stars from Gaia are shown as red circles. TOI-510 is marked by a white cross. The _TESS_ photometric aperture is highlighted in red. (*fig:tpf510*)

</div>
<div id="div_fig2">

<img src="tmp_2501.14355/./combined_rvphaseplot_041224.png" alt="Fig20" width="100%"/>

**Figure 20. -** Radial velocity data for TOI-271, TOI-641, TOI-697 and TOI-745, phase folded on the period from the joint model. Random models from the MCMC trace are shown in orange. The model best matching the 95\% confidence limit on semi-amplitude is shown in red to highlight the limit we place on candidate planet mass. (*fig:rvnulls*)

</div>
<div id="div_fig3">

<img src="tmp_2501.14355/./SR_DM_NCORESonly_080724.png" alt="Fig11" width="100%"/>

**Figure 11. -** Planets with known mass and radius determined to better than $3\sigma$ plotted in Insolation-Radius space, coloured and inversely scaled in size by their density. Only planets with host star mass $M_*>0.7M_\odot$ are shown. The radius gap location determined in \citet{Ho:2023ab} is shown as a black dashed line. The background is coloured by a kernel density estimate of the planets shown. The diversity of sub-Neptune composition above the gap can be seen in the range of densities observed in this region. (*fig:SRplot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.14355"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mass}{\Sigma_{\star}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$\/ preparation: Extracting physical parameters from galaxies with machine learning

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.14408-b31b1b.svg)](https://arxiv.org/abs/2501.14408)<mark>Appeared on: 2025-01-27</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ \/ mission is generating a vast amount of imaging data in four broadband filters at high angular resolution. This will allow the detailed study of mass, metallicity, and stellar populations across galaxies, which will constrain their formation and evolutionary pathways. Transforming the $\Euclid$ \/ imaging for large samples of galaxies into maps of physical parameters in an efficient and reliable manner is an outstanding challenge. We investigate the power and reliability of machine learning techniques to extract the distribution of physical parameters within well-resolved galaxies. We focus on estimating stellar mass surface density, mass-averaged stellar metallicity and age. We generate noise-free, synthetic high-resolution ( $100{\rm   pc} \times100 {\rm   pc}$ ) imaging data in the $\Euclid$ \/ photometric bands for a set of 1154 galaxies from the TNG50 cosmological simulation. The images are generated with the SKIRT radiative transfer code, taking into account the complex 3D distribution of stellar populations and interstellar dust attenuation. We use a machine learning framework to map the idealised mock observational data to the physical parameters on a pixel-by-pixel basis. We find that stellar mass surface density can be accurately recovered with a $\leq 0.130 {\rm  dex}$ scatter.Conversely, stellar metallicity and age estimates are, as expected, less robust, but still contain significant information which originates from underlying correlations at a sub-kpc scale between stellar mass surface density and stellar population properties. As a corollary, we show that TNG50 follows a spatially resolved mass-metallicity relation that is consistent with observations. Due to its relatively low computational and time requirements, that is a time-frame of minutes without dedicated high performance computing infrastructure once it has been trained, our method allows for fast and robust estimates of the stellar mass surface density distributions of nearby galaxies from four-filter $\Euclid$ \/ imaging data. Equivalent estimates of stellar population properties (stellar metallicity and age) are less robust but still hold value as first-order approximations across large samples.

</div>

<div id="div_fig1">

<img src="tmp_2501.14408/./Figures/models_all.png" alt="Fig4" width="100%"/>

**Figure 4. -** Predictions for $\mass$(top), $Z_{\star}$(middle), and $t_{\star}$(bottom) using the four \Euclid\/ bands as the input features for the three models: linear regression (left), random forest (middle), and deep neural network (right) as a function of their true value. Diagonal line marks the identity, the additional two dashed lines mark the limit for the catastrophic outliers (instances where the predicted value is two times larger or smaller than the true value). Running medians, with their 16$^\text{th}$ and 84$^\text{th}$ percentiles are marked with large blue points with error bars. (*fig:models_all*)

</div>
<div id="div_fig2">

<img src="tmp_2501.14408/./Figures/histograms_tot.png" alt="Fig3" width="100%"/>

**Figure 3. -** The normalised distribution of $\logten M_{\star}$, $\logten Z_{\star}$, and $\logten t_{\star}$ global values of the galaxy properties for the entire data set, training set, and test set (see legend). (*fig:hist*)

</div>
<div id="div_fig3">

<img src="tmp_2501.14408/./Figures/TNG539333_O5_good.png" alt="Fig6" width="100%"/>

**Figure 6. -** Full disc map of galaxy TNG 539333 (O5 projection) for $\logten \left[$\mass$ /{\rm \left(M_{\odot} pc^{-2}\right)}\right]$(left), $\logten \left(Z_{\star} / {\rm Z_{\odot}}\right) $(middle), and $\logten \left(t_{\star} / {\rm Gyr}\right)$(right) as an example of a more accurate model. From top to bottom, top row shows the actual values for each property, followed by the model created by a deep neural network algorithm on four \Euclid\/ bands, and the third row shows the residuals, where points in red (blue) represent pixels where the actual values are higher (lower) than those in the model. Histograms contained in the lower right corner of the residual maps show the distribution of error prediction ($\logten$$y_i$ - $\logten$$\hat{y}_i$, where $y_i$ is the actual $i$-th value, and $\hat{y}_i$ is the predicted $i$-th value of the pixel), dashed lines mark the 16$^\text{th}$ and 84$^\text{th}$ percentile of the distribution for the pixels contained within the dashed circle of a 12 kpc radius. Bottom row shows radial profiles for the true (red) and predicted (blue) values where large points mark the median, and error bars mark the 16$^\text{th}$ and 84$^\text{th}$ percentile of the distribution contained within the radial bin, as well as their residuals plotted underneath. (*fig:galaxy_good*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.14408"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\photmoh}{[M/H]_{\rm phot}}$
$\newcommand{\teff}{T_{\mathrm{eff}}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\moh}{[M/H]}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\singlechi}{\chi^2_{\mathrm{single}}}$
$\newcommand{\binarychi}{\chi^2_{\mathrm{WDMS}}}$
$\newcommand{\jd}[1]{\textcolor{blue}{JD:~#1}}$
$\newcommand{\hw}[1]{\textcolor{purple}{HW:~#1}}$
$\newcommand{\gaia}{\textsl{Gaia}}$</div>



<div id="title">

# Identification of 30,000 White Dwarf-Main Sequence binaries candidates from Gaia DR3 BP/RP(XP) low-resolution spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.14494-b31b1b.svg)](https://arxiv.org/abs/2501.14494)<mark>Appeared on: 2025-01-27</mark> -  _30 pages, 12 figures, Submitted to ApJS_

</div>
<div id="authors">

<mark>J. Li</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>J. Müller-Horn</mark>, <mark>R. Seeburger</mark>

</div>
<div id="abstract">

**Abstract:** White dwarf-main sequence (WDMS) binary systems are essential probes for understanding binary stellar evolution and play a pivotal role in constraining theoretical models of various transient phenomena.In this study, we construct a catalog of WDMS binaries using Gaia DR3's low-resolution BP/RP (XP) spectra.Our approach integrates a model-independent neural network for spectral modelling with Gaussian Process Classification to accurately identify WDMS binaries among over 10 million stars within 1 kpc.This study identify approximately 30,000 WDMS binary candidates, including 1,700 high-confidence systems confirmed through spectral fitting.Our technique is shown to be effective at detecting systems where the main-sequence star dominates the spectrum - cases that have historically challenged conventional methods.Validation using GALEX photometry reinforces the reliability of our classifications: 70 \% of candidates with an absolute magnitude $M_{G} > 7$ exhibit UV excess, a characteristic signature of white dwarf companions.Our all-sky catalog of WDMS binaries expands the available dataset for studying binary evolution and white dwarf physics and sheds light on the formation of WDMS.

</div>

<div id="div_fig1">

<img src="tmp_2501.14494/./fig_wdms/HRD_wdmsfit.png" alt="Fig6" width="100%"/>

**Figure 6. -** CMD showing the distribution of WDMS binary candidates identified through $\chi^2$ fitting of XP spectra (red dots).
    The background grayscale shows the density distribution of all stars, with the scale bar indicating the logarithmic number of stars per bin.
    The x-axis shows the de-reddened $\gaia$ color (B-R)$_0$, while the y-axis shows the absolute G magnitude ($M_{G0}$) after extinction correction.
    The 10 cyan stars are examples of the $\chi^2$ selected WDMS binaries, the yellow lines show the best fit spectral decomposition of a WD and an MS by XP spectra.
    The binary candidates primarily occupy the region between the main sequence (right) and white dwarf sequence (left), as expected for composite systems.
    The blue dotted line denotes the requirements of the WD component of a binary model fit, which should lie to the left of this line as in the WD regime. (*fig:hrd_wdmsfit*)

</div>
<div id="div_fig2">

<img src="tmp_2501.14494/./fig_wdms/camd.png" alt="Fig3.1" width="33%"/><img src="tmp_2501.14494/./fig_wdms/camd_ext.png" alt="Fig3.2" width="33%"/><img src="tmp_2501.14494/./fig_wdms/camd_1stclean.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** 
    Left: Color-magnitude diagram (CMD) showing the density distribution of stars of selected $\gaia$  data, with absolute G-band magnitude ($M_G$) plotted against $B_p$-$R_p$ color (B-R).
    Middle: Extinction-corrected CMD for the stars displayed in the left panel, accounting for interstellar reddening.
    Right: The CMD showing the distribution of stars in our sample within $\sim$300 pc that passed our initial cleaning for training the spectral emulator.
    The blue dotted line represents a PARSEC isochrone for [M/H]$= 0.5$,  which serves as an upper boundary for selecting single main sequence stars.
    The blue dashed line is a hard cut to select white dwarf stars.  (*fig:initial query*)

</div>
<div id="div_fig3">

<img src="tmp_2501.14494/./fig_wdms/HRD_galex.png" alt="Fig11" width="100%"/>

**Figure 11. -** 
   Comparison of WDMS binary candidates selected by different methods in various CMDs.
   Left panel: $\gaia$ CMD showing absolute G magnitude versus de-reddened B-R color.
   Middle and right panels: GALEX-Gaia CMD using NUV and FUV bands, respectively.
   In all panels, blue dots represent WDMS binaries identified through $\chi^2$-fitting of their XP spectra (\texttt{Chi2} sample in Table \ref{tab:sample_stats}), while red dots show WDMS candidates identified by our Gaussian Process Classifier (GPC) (\texttt{QC} sample) but missed by the \texttt{Chi2} sample.
   Yellow dots mark possible MSMS binaries (\texttt{flag$_{\rm MSMS}$}=1), mostly brighter than the dotted line at $M_{G0} = 7$.
   The underlying grey points show the single stars, as defined by the GPC with $P_{\rm WDMS}<0.2$.
    (*fig:galex*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.14494"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

318  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
